In [ ]:
/root/hackerday/12_employee_access_need/train.csv
/root/hackerday/12_employee_access_need/test.csv
#/root/hackerday/12_employee_access_need/sampleSubmission.csv

In [64]:
library(irlba)
library(gbm)
library(randomForest)
library(glmnet)

ERROR: Error in library(sqldf): there is no package called ‘sqldf’


In [69]:
library(sqldf)

ERROR: Error in library(sqldf): there is no package called ‘sqldf’


In [12]:
train <- read.csv("/root/hackerday/12_employee_access_need/train.csv")

In [13]:
head(train)

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
1,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
2,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
3,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
4,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
5,1,42680,5905,117929,117930,119569,119323,123932,19793,119325
6,0,45333,14561,117951,117952,118008,118568,118568,19721,118570


In [14]:
test <- read.csv("/root/hackerday/12_employee_access_need/test.csv")

In [15]:
head(test)

,id,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
1,1,78766,72734,118079,118080,117878,117879,118177,19721,117880
2,2,40644,4378,117961,118327,118507,118863,122008,118398,118865
3,3,75443,2395,117961,118300,119488,118172,301534,249618,118175
4,4,43219,19986,117961,118225,118403,120773,136187,118960,120774
5,5,42093,50015,117961,118343,119598,118422,300136,118424,118425
6,6,44722,1755,117961,117962,119223,125793,146749,118643,125795


In [9]:
names(trd)

[1] "ACTION"           "RESOURCE"         "MGR_ID"           "ROLE_ROLLUP_1"   
 [5] "ROLE_ROLLUP_2"    "ROLE_DEPTNAME"    "ROLE_TITLE"       "ROLE_FAMILY_DESC"
 [9] "ROLE_FAMILY"      "ROLE_CODE"

In [54]:
dim(train)
dim(test)

[1] 32769    11

[1] 58921    11

In [55]:
train$id<-100000+c(1:dim(train)[1])
test$ACTION<-(-1)
total<-rbind(train,test)
head(total)

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,id
1,1,39353,85475,117961,118300,123472,117905,117906,290919,117908,100001
2,1,17183,1540,117961,118343,123125,118536,118536,308574,118539,100002
3,1,36724,14457,118219,118220,117884,117879,267952,19721,117880,100003
4,1,36135,5396,117961,118343,119993,118321,240983,290919,118322,100004
5,1,42680,5905,117929,117930,119569,119323,123932,19793,119325,100005
6,0,45333,14561,117951,117952,118008,118568,118568,19721,118570,100006


In [37]:
total$ACTION0<-total$ACTION
total$ACTION<-(1-total$ACTION0)
dim(total)

[1] 91690    16

In [36]:
total$y<-total$ACTION
total$y[total$y>1]<-0
total$ws <- 1
total$ws[total$ACTION0<0]<-0
total$dummy<-'A'
total$split1<-0
total$split1[total$ACTION0<0]<-2

In [70]:
#data transformation
#convert the categorical variables into factor variables so that we can use those factors as features in our model
total$res_f<-factor(total$RESOURCE)
total$res_f_id<-as.integer(total$res_f)
total$mgr_f<-factor(total$MGR_ID)
total$mgr_f_id<-as.integer(total$mgr_f)
total$rocd_f<-factor(total$ROLE_CODE)
total$rocd_f_id<-as.integer(total$rocd_f)
total$rr1_f<-factor(total$ROLE_ROLLUP_1)
total$rr1_f_id<-as.integer(total$rr1_f)
total$rr2_f<-factor(total$ROLE_ROLLUP_2)
total$rr2_f_id<-as.integer(total$rr2_f)
total$rt_f<-factor(total$ROLE_TITLE)
total$rt_f_id<-as.integer(total$rt_f)
total$rf_f<-factor(total$ROLE_FAMILY)
total$rf_f_id<-as.integer(total$rf_f)
total$rd_f<-factor(total$ROLE_DEPTNAME)
total$rd_f_id<-as.integer(total$rd_f)
total$rfd_f<-factor(total$ROLE_FAMILY_DESC)
total$rfd_f_id<-as.integer(total$rfd_f)

total$mgr_res_cnt<-my.f2cnt(total,"MGR_ID","RESOURCE")

total$mgr_res_f<-as.factor(total$MGR_ID*10000+total$RESOURCE)
total$mgr_res_f_id<-as.integer(total$mgr_res_f)

ERROR: Error in my.f2cnt(total, "MGR_ID", "RESOURCE"): could not find function "sqldf"


In [62]:
#1 way count
mean_t<-with(total[total$split1==0,],sum(ACTION)*1.0/length(ACTION))
for (ii in 2:10){
    print(names(total)[ii])
    total$x<-total[,ii]
    sum1<-sqldf("select x, sum(1) as cnt
               from total group by 1")
    tmp<-sqldf("select cnt from total a left join sum1 b on a.x=b.x")
    total[,paste(names(total)[ii],"_cnt",sep="")]<-tmp$cnt
}

[1] "RESOURCE"


ERROR: Error: could not find function "sqldf"


In [60]:
#2 way count
my.f2cnt<-function(total2,vn1,vn2,filter=T){
    df<-data.frame(f1=total2[,vn1],f2=total2[,vn2],filter=filter)
    sum1<-sqldf("select f1, f2, count(*) as cnt from df where filter=1 group by 1,2")
    tmp<-sqldf("select b.cnt from df a left join sum1 b on a.f1=b.f1 and a.f2=b.f2")
    tmp$cnt[is.na(tmp$cnt)]<-0
    return(tmp$cnt)
}

In [61]:
#3way count
my.f3cnt<-function(total2,vn1,vn2,vn3, filter=T){
    df<-data.frame(f1=total2[,vn1],f2=total2[,vn2],f3=total2[,vn3], filter=filter)
    sum1<-sqldf("select f1, f2, f3, count(*) as cnt from df where filter=1 group by 1,2, 3")
    tmp<-sqldf("select b.cnt from df a left join sum1 b on a.f1=b.f1 and a.f2=b.f2 and a.f3=b.f3")
    tmp$cnt[is.na(tmp$cnt)]<-0
    return(tmp$cnt)
}

In [ ]:
#selected 2 way count for feature creation
total$mgr_rocd_cnt<-my.f2cnt(total,"MGR_ID","ROLE_CODE")
total$mgr_rd_cnt<-my.f2cnt(total,"MGR_ID","ROLE_DEPTNAME")
total$mgr_rf_cnt<-my.f2cnt(total,"MGR_ID","ROLE_FAMILY")
total$mgr_rfd_cnt<-my.f2cnt(total,"MGR_ID","ROLE_FAMILY_DESC")
total$mgr_rt_cnt<-my.f2cnt(total,"MGR_ID","ROLE_TITLE")
total$mgr_rr1_cnt<-myf2cnt(total,"MGR_ID","ROLE_ROLLUP_1")
total$mgr_rr2_cnt<-myf2cnt(total,"MGR_ID","ROLE_ROLLUP_2")

total$res_rf_cnt<-my.f2cnt(total,"RESOURCE","ROLE_FAMILY")
total$res_rfd_cnt<-my.f2cnt(total,"RESOURCE","ROLE_FAMILY_DESC")
total$res_rt_cnt<-my.f2cnt(total,"RESOURCE","ROLE_TITLE")
total$res_rd_cnt<-my.f2cnt(total,"RESOURCE","ROLE_DEPTNAME")
total$res_rocd_cnt<-my.f2cnt(total,"RESOURCE","ROLE_CODE")
total$res_rr1_cnt<-myf2cnt(total,"RESOURCE","ROLE_ROLLUP_1")
total$res_rr2_cnt<-myf2cnt(total,"RESOURCE","ROLE_ROLLUP_2")

total$rr1_rf_cnt<-my.f2cnt(total,"ROLE_ROLLUP_1","ROLE_FAMILY")
total$rr1_rfd_cnt<-my.f2cnt(total,"ROLE_ROLLUP_1","ROLE_FAMILY_DESC")
total$rr1_rt_cnt<-my.f2cnt(total,"ROLE_ROLLUP_1","ROLE_TITLE")
total$rr1_rd_cnt<-my.f2cnt(total,"ROLE_ROLLUP_1","ROLE_DEPTNAME")
total$rr1_rocd_cnt<-my.f2cnt(total,"ROLE_ROLLUP_1","ROLE_CODE")
total$rr1_rr2_cnt<-myf2cnt(total,"ROLE_ROLLUP_1","ROLE_ROLLUP_2")

total$rr2_rf_cnt<-my.f2cnt(total,"ROLE_ROLLUP_2","ROLE_FAMILY")
total$rr2_rfd_cnt<-my.f2cnt(total,"ROLE_ROLLUP_2","ROLE_FAMILY_DESC")
total$rr2_rt_cnt<-my.f2cnt(total,"ROLE_ROLLUP_2","ROLE_TITLE")
total$rr2_rd_cnt<-my.f2cnt(total,"ROLE_ROLLUP_2","ROLE_DEPTNAME")
total$rr2_rocd_cnt<-my.f2cnt(total,"ROLE_ROLLUP_2","ROLE_CODE")

total$rf_rfd_cnt<-my.f2cnt(total,"ROLE_FAMILY","ROLE_FAMILY_DESC")
total$rf_rt_cnt<-my.f2cnt(total,"ROLE_FAMILY","ROLE_TITLE")
total$rf_rd_cnt<-my.f2cnt(total,"ROLE_FAMILY","ROLE_DEPTNAME")
total$rf_rocd_cnt<-my.f2cnt(total,"ROLE_FAMILY","ROLE_CODE")

total$rfd_rt_cnt<-my.f2cnt(total,"ROLE_FAMILY_DESC","ROLE_TITLE")
total$rfd_rd_cnt<-my.f2cnt(total,"ROLE_FAMILY_DESC","ROLE_DEPTNAME")
total$rfd_rocd_cnt<-my.f2cnt(total,"ROLE_FAMILY_DESC","ROLE_CODE")

total$rt_rd_cnt<-my.f2cnt(total,"ROLE_TITLE","ROLE_DEPTNAME")
total$rt_rocd_cnt<-my.f2cnt(total,"ROLE_TITLE","ROLE_CODE")

total$rd_rocd_cnt<-my.f2cnt(total,"ROLE_DEPTNAME","ROLE_CODE")
total$rd_rf_cnt<-my.f2cnt(total,"ROLE_DEPTNAME","ROLE_FAMILY")


In [ ]:
#3 way count based feature creation for classification based models
total$res_mgr_rr2_cnt<-myf3cnt(total,"RESOURCE","MGR_ID","ROLE_ROLLUP_2")
total$res_mgr_rd_cnt<-myf3cnt(total,"RESOURCE","MGR_ID","ROLE_DEPTNAME")
total$res_mgr_rcod_cnt<-myf3cnt(total,"RESOURCE","MGR_ID","ROLE_CODE")
total$res_mgr_rdf_cnt<-myf3cnt(total,"RESOURCE","MGR_ID","ROLE_FAMILY_DESC")
total$res_mgr_rf_cnt<-myf3cnt(total,"RESOURCE","MGR_ID","ROLE_FAMILY")
total$rf_mgr_rd_cnt<-myf3cnt(total,"ROLE_FAMILY","MGR_ID","ROLE_DEPTNAME")
total$rf_mgr_rr2_cnt<-myf3cnt(total,"ROLE_FAMILY","MGR_ID","ROLE_ROLLUP_2")
total$res_rd_rf_cnt<-myf3cnt(total,"RESOURCE","ROLE_DEPTNAME","ROLE_FAMILY")
total$res_rd_rr2_cnt<-myf3cnt(total,"RESOURCE","ROLE_DEPTNAME","ROLE_ROLLUP_2")

In [ ]:
#define one id which should show the combnation of role related variables
sum1<-sqldf("select role_rollup_1, role_rollup_2, role_title, role_deptname, role_family, role_family_desc, role_code, count(*) as cnt from total group by 1,2,3,4,5,6,7")

sum1$role_id <- c(1:dim(sum1)[1])

tmp<-sqldf("select role_id, b.cnt from total a join sum1 b on a.role_rollup_1=b.role_rollup_1 and 
a.role_rollup_2=b.role_rollup_2 and a.role_title=b.role_title and a.role_deptname=b.role_deptname and a.role_family=b.role_family
and a.role_family_desc=b.role_family_desc and a.role_code=b.role_code")

total$rid<-tmp$role_id

total$rid_res_cnt<-my.f2cnt(total,"rid","RESOURCE")
total$rid_cnt<-my.f2cnt(total,"rid","dummy")

total$all_but_res_cnt<-my.f2cnt(total,"rid","MGR_ID")

In [ ]:
#role type id, this is a combination of role related variables
sum1<-sqldf("select role_rollup_1, role_rollup_2, role_title, role_deptname, role_family, count(*) as cnt from total group by 1,2,3,4,5")

sum1$roletype_id <- c(1:dim(sum1)[1])

tmp<-sqldf("select roletype_id, b.cnt from total a join sum1 b on a.role_rollup_1=b.role_rollup_1 and 
a.role_rollup_2=b.role_rollup_2 and a.role_title=b.role_title and a.role_deptname=b.role_deptname and a.role_family=b.role_family")

total$rtype_id<- tmp$roletype_id


total$rtype_cnt<-my.f2cnt(total,"rtype_id","dummy")
total$all_but_rfd_cnt<-my.f2cnt(total,"mgr_res_f_id","rtype_id")

In [ ]:
#feature extraction using eigen values using singular value decomposition method
sum1<-sqldf("select rid, res_f_id, sum(1) as cnt from total group by 1, 2")

sm1<-sparseMatrix(i=sum1$res_f_id, j=sum1$rid, x=1)

svd1<-irlba(sm1, nu=5, nv=5, adjust=3)

total$rid_ev1<-svd1$v[total$rid,1]
total$rid_ev2<-svd1$v[total$rid,2]
total$rid_ev3<-svd1$v[total$rid,3]
total$rid_ev4<-svd1$v[total$rid,4]
total$rid_ev5<-svd1$v[total$rid,5]

total$res_ev1<-svd1$u[total$res_f_id,1]
total$res_ev2<-svd1$u[total$res_f_id,2]
total$res_ev3<-svd1$u[total$res_f_id,3]
total$res_ev4<-svd1$u[total$res_f_id,4]
total$res_ev5<-svd1$u[total$res_f_id,5]


sum1<-sqldf("select rid, mgr_f_id, sum(1) as cnt from total group by 1, 2")

sm1<-sparseMatrix(i=sum1$mgr_f_id, j=sum1$rid, x=1)

svd1<-irlba(sm1, nu=5, nv=5, adjust=3)

total$rid_mgr_ev1<-svd1$v[total$rid,1]
total$rid_mgr_ev2<-svd1$v[total$rid,2]
total$rid_mgr_ev3<-svd1$v[total$rid,3]
total$rid_mgr_ev4<-svd1$v[total$rid,4]
total$rid_mgr_ev5<-svd1$v[total$rid,5]

total$mgr_rid_ev1<-svd1$u[total$mgr_f_id,1]
total$mgr_rid_ev2<-svd1$u[total$mgr_f_id,2]
total$mgr_rid_ev3<-svd1$u[total$mgr_f_id,3]
total$mgr_rid_ev4<-svd1$u[total$mgr_f_id,4]
total$mgr_rid_ev5<-svd1$u[total$mgr_f_id,5]

sum1<-sqldf("select res_f_id, mgr_f_id, sum(1) as cnt from total group by 1, 2")

sm1<-sparseMatrix(i=sum1$mgr_f_id, j=sum1$res_f_id, x=1)

svd1<-irlba(sm1, nu=5, nv=5, adjust=3)

total$res_mgr_ev1<-svd1$v[total$res_f_id,1]
total$res_mgr_ev2<-svd1$v[total$res_f_id,2]
total$res_mgr_ev3<-svd1$v[total$res_f_id,3]
total$res_mgr_ev4<-svd1$v[total$res_f_id,4]
total$res_mgr_ev5<-svd1$v[total$res_f_id,5]

total$mgr_res_ev1<-svd1$u[total$mgr_f_id,1]
total$mgr_res_ev2<-svd1$u[total$mgr_f_id,2]
total$mgr_res_ev3<-svd1$u[total$mgr_f_id,3]
total$mgr_res_ev4<-svd1$u[total$mgr_f_id,4]
total$mgr_res_ev5<-svd1$u[total$mgr_f_id,5]


In [ ]:
#randomized leave-one-out average actual for categorical variables
my_exp1<-function(d1, vn1, vn2, y, vnp, filter, cred_k, r_k=.3){
  d2<-d1[, c(vn1, vn2, y, vnp)]
  names(d2)<-c("f1", "f2", "a", "p")
  d2$filter<-filter
  sum1<-sqldf("select f1, f2, sum(1) as cnt, sum(p) as sump, sum(a) as suma from d2 where filter=1 group by 1,2")
  tmp1<-sqldf("select a.p, b.cnt, b.sump, b.suma from d2 a left join sum1 b on a.f1=b.f1 and a.f2=b.f2")
  tmp1$cnt[is.na(tmp1$cnt)]<-0
  tmp1$avgp<-with(tmp1, sump/cnt)
  tmp1$avgp[is.na(tmp1$avgp)]<-0
  tmp1$suma[is.na(tmp1$suma)]<-0
  tmp1$cnt[filter]<-tmp1$cnt[filter]-1
  tmp1$suma[filter]<-tmp1$suma[filter]-d1$y[filter]
  tmp1$exp_a<-with(tmp1, suma/cnt)
  tmp1$adj_a<-with(tmp1, (suma+p*cred_k)/(cnt+cred_k))
  tmp1$exp_a[is.na(tmp1$exp_a)]<-tmp1$p[is.na(tmp1$exp_a)]
  tmp1$adj_a[is.na(tmp1$adj_a)]<-tmp1$p[is.na(tmp1$adj_a)]
  tmp1$adj_a[filter]<-tmp1$adj_a[filter]*(1+(runif(sum(filter))-0.5)*r_k)
  return(tmp1)
}


In [ ]:
#calc exps for selected one way and 2 ways
tv_spec_trans<-function(total, calc_pred=T) {
  
  
  if(calc_pred) {
    total$pred0<-with(total[total$split1==0,], mean(ACTION*1.0))
    total$pred0_offset<-0
  } else {
    total$pred0_offset<-total$pred0
    
  }
  total$dummy<-1

In [ ]:
t1<-my_exp1(total, "MGR_ID", "rid", "ACTION", "pred0", total$split1==0, 40, r_k=r_k)
  total$exp_all_but_res<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "dummy", "rid", "ACTION", "pred0", total$split1==0, 40, r_k=r_k)
  total$exp_rid<-t1$adj_a-total$pred0_offset
  
  
  t1<-my_exp1(total, "MGR_ID", "dummy", "ACTION", "pred0", total$split1==0, 40, r_k=r_k)
  total$exp_mgr<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "RESOURCE", "dummy", "ACTION", "pred0", total$split1==0, 40, r_k=r_k)
  total$exp_res<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "ROLE_FAMILY_DESC", "dummy", "ACTION", "pred0", total$split1==0, 40, r_k=r_k)
  total$exp_rfd<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "RESOURCE", "ROLE_DEPTNAME", "ACTION", "pred0", total$split1==0, 40, r_k=r_k)
  total$exp_res_rd<-t1$adj_a-total$pred0_offset
  
  
  t1<-my_exp1(total, "ROLE_TITLE", "dummy", "ACTION", "pred0", total$split1==0, 20, r_k=r_k)
  total$exp_rt<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "ROLE_ROLLUP_1", "dummy", "ACTION", "pred0", total$split1==0, 20, r_k=r_k)
  total$exp_rr1<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "ROLE_ROLLUP_2", "dummy", "ACTION", "pred0", total$split1==0, 20, r_k=r_k)
  total$exp_rr2<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "ROLE_FAMILY", "dummy", "ACTION", "pred0", total$split1==0, 20, r_k=r_k)
  total$exp_rf<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "ROLE_CODE", "dummy", "ACTION", "pred0", total$split1==0, 20, r_k=r_k)
  total$exp_rocd<-t1$adj_a-total$pred0_offset
  
  
  t1<-my_exp1(total, "ROLE_DEPTNAME", "dummy", "ACTION", "pred0", total$split1==0, 20, r_k=r_k)
  total$exp_rd<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "MGR_ID", "RESOURCE", "ACTION", "pred0", total$split1==0, 40, r_k=r_k)
  total$exp_mgr_res<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "MGR_ID", "ROLE_TITLE", "ACTION", "pred0", total$split1==0, 40, r_k=r_k)
  total$exp_mgr_rt<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "MGR_ID", "ROLE_FAMILY_DESC", "ACTION", "pred0", total$split1==0, 40, r_k=r_k)
  total$exp_mgr_rfd<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "RESOURCE", "ROLE_ROLLUP_2", "ACTION", "pred0", total$split1==0, 40, r_k=r_k)
  total$exp_res_rr2<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "RESOURCE", "ROLE_TITLE", "ACTION", "pred0", total$split1==0, 40, r_k=r_k)
  total$exp_res_rt<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "RESOURCE", "rid", "ACTION", "pred0", total$split1==0, 40, r_k=r_k)
  total$exp_res_rid<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "ROLE_FAMILY", "ROLE_ROLLUP_2", "ACTION", "pred0", total$split1==0, 40, r_k=r_k)
  total$exp_rf_rr2<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "ROLE_DEPTNAME", "ROLE_ROLLUP_2", "ACTION", "pred0", total$split1==0, 40, r_k=r_k)
  total$exp_rd_rr2<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "ROLE_DEPTNAME", "rfd_f", "ACTION", "pred0", total$split1==0, 40, r_k=r_k)
  total$exp_rd_rfd<-t1$adj_a-total$pred0_offset
  
  t1<-my_exp1(total, "rfd_f", "rocd_f", "ACTION", "pred0", total$split1==0, 40, r_k=r_k)
  total$exp_rfd_rocd<-t1$adj_a-total$pred0_offset
  
  
  return(total)
}

In [ ]:
#model specification for tree based models 
mdl_def_tree<-paste("RESOURCE+ROLE_ROLLUP_1+ROLE_ROLLUP_2+ROLE_DEPTNAME+ROLE_TITLE",
                    "+ROLE_FAMILY_DESC+ROLE_FAMILY+MGR_ID+ROLE_CODE",
                    
                    "+RESOURCE_cnt+ROLE_ROLLUP_1_cnt+ROLE_ROLLUP_2_cnt+ROLE_DEPTNAME_cnt+ROLE_TITLE_cnt",
                    "+ROLE_FAMILY_DESC_cnt+ROLE_FAMILY_cnt+MGR_ID_cnt+mgr_res_cnt+ROLE_CODE_cnt",
                    
                    "+res_rf_cnt+res_rfd_cnt+res_rt_cnt+res_rr1_cnt+res_rr2_cnt+res_rd_cnt",
                    "+mgr_rf_cnt+mgr_rfd_cnt+mgr_rt_cnt+mgr_rr1_cnt+mgr_rr2_cnt+mgr_rd_cnt",
                    "+res_mgr_rr2_cnt+res_mgr_rd_cnt+res_rd_rr2_cnt",
                    "+rr1_rr2_cnt+rr1_rf_cnt+rr1_rfd_cnt+rr1_rt_cnt+rr1_rd_cnt",
                    "+rr2_rf_cnt+rr2_rfd_cnt+rr2_rt_cnt+rr2_rd_cnt",
                    "+rf_rfd_cnt+rf_rt_cnt+rf_rd_cnt+rfd_rt_cnt+rfd_rd_cnt",
                    "+rt_rd_cnt+rid_res_cnt",
                    
                    "+exp_res+exp_mgr+exp_rfd+exp_res_rd + exp_res_rt + exp_res_rr2",
                    "+exp_mgr_res + exp_mgr_rt + exp_mgr_rfd +all_but_res_cnt +exp_rid",
                    "+exp_res_rid+ exp_all_but_res"
                    ,
                    
                    "+rid_ev1+rid_ev2+rid_ev3+res_ev1+res_ev2+res_ev3+rid_ev4+rid_ev5+res_ev4+res_ev5",
                    "+rid_mgr_ev1+rid_mgr_ev2+rid_mgr_ev3+mgr_rid_ev1+mgr_rid_ev2+mgr_rid_ev3+rid_mgr_ev4+rid_mgr_ev5+mgr_rid_ev4+mgr_rid_ev5",
                    "+mgr_res_ev1+mgr_res_ev2+mgr_res_ev3+res_mgr_ev1+res_mgr_ev2+res_mgr_ev3+mgr_res_ev4+mgr_res_ev5+res_mgr_ev4+res_mgr_ev5"
)

In [ ]:
#model specification for glmnet
mdl_def_gnet<-paste("y~log(exp_all_but_res)+log(exp_res)+log(exp_rfd)+log(exp_res_rd)+log(exp_rd+exp_mgr)+log(exp_rid)",
                    "+log(exp_mgr_res)+log(exp_rr2)+log(exp_all_but_res)*log(exp_rid)",
                    "+log(exp_rid)+log(exp_rid)*log(exp_rid)+log(exp_res_rr2)+log(exp_res_rr2)*log(exp_res_rr2)",
                    "+log(exp_mgr_rt)+log(exp_mgr_rt)*log(exp_mgr_rt)+log(exp_res_rd)+log(exp_res_rd)*log(exp_res_rd)",
                    "+log(exp_mgr)+log(exp_mgr)*log(exp_mgr)+log(exp_mgr_rfd)+log(exp_mgr_rfd)*log(exp_mgr_rfd)",
                    "+log(exp_rf)+log(exp_rocd)+log(exp_res_rt)+log(exp_res_rid)+log(exp_res_rr2)+log(exp_res_rt)",
                    "+log(res_rd_cnt-0.5)+log(rid_res_cnt)+log(mgr_rt_cnt)",
                    "+log(exp_all_but_res/(1-exp_all_but_res))+log(exp_all_but_res)*log(exp_all_but_res)",
                    "+log(exp_res)*log(exp_res) + log(exp_res)*log(exp_all_but_res) +log(res_rr2_cnt)",
                    "+log(res_rd_cnt-0.5)*log(rid_res_cnt)",
                    "+log(RESOURCE_cnt)++log(RESOURCE_cnt)*+log(RESOURCE_cnt) +log(all_but_res_cnt)",
                    "+log(mgr_res_cnt)+log(mgr_res_cnt)*log(mgr_res_cnt)",
                    "+log(rt_rd_cnt)+log(rt_rd_cnt)*log(rt_rd_cnt)",
                    "+log(mgr_rd_cnt)+log(mgr_rd_cnt)*log(mgr_rd_cnt)",
                    "+log(ROLE_TITLE_cnt)+log(ROLE_TITLE_cnt)*log(ROLE_TITLE_cnt)",
                    "+log(MGR_ID_cnt)+log(MGR_ID_cnt)*log(MGR_ID_cnt)",
                    "+log(ROLE_DEPTNAME_cnt)+log(ROLE_DEPTNAME_cnt)*log(ROLE_DEPTNAME_cnt)",
                    "+log(ROLE_ROLLUP_2_cnt)+log(ROLE_FAMILY_cnt)+log(ROLE_FAMILY_DESC_cnt)+log(ROLE_ROLLUP_1_cnt)",
                    "+log(ROLE_CODE_cnt)+log(rr2_rd_cnt)",
                    "+log(res_mgr_rr2_cnt)+log(rf_rt_cnt)+log(res_rfd_cnt)+log(rf_rt_cnt)",
                    "+log(exp_rd_rr2)",
                    "+rid_ev1+rid_ev2+rid_ev3+res_ev1+res_ev2+res_ev3+rid_ev4+rid_ev5+res_ev4+res_ev5",
                    "+rid_mgr_ev1+rid_mgr_ev2+rid_mgr_ev3+mgr_rid_ev1+mgr_rid_ev2+mgr_rid_ev3+rid_mgr_ev4+rid_mgr_ev5+mgr_rid_ev4+mgr_rid_ev5",
                    "+mgr_res_ev1+mgr_res_ev2+mgr_res_ev3+res_mgr_ev1+res_mgr_ev2+res_mgr_ev3+mgr_res_ev4+mgr_res_ev5+res_mgr_ev4+res_mgr_ev5"
)

In [ ]:
#model specification for tree based models without exp variables
mdl_def_tree_noexp<-paste("RESOURCE+ROLE_ROLLUP_1+ROLE_ROLLUP_2+ROLE_DEPTNAME+ROLE_TITLE",
                          "+ROLE_FAMILY_DESC+ROLE_FAMILY+MGR_ID+ROLE_CODE",
                          
                          "+RESOURCE_cnt+ROLE_ROLLUP_1_cnt+ROLE_ROLLUP_2_cnt+ROLE_DEPTNAME_cnt+ROLE_TITLE_cnt",
                          "+ROLE_FAMILY_DESC_cnt+ROLE_FAMILY_cnt+MGR_ID_cnt+mgr_res_cnt+ROLE_CODE_cnt",
                          
                          "+res_rf_cnt+res_rfd_cnt+res_rt_cnt+res_rr1_cnt+res_rr2_cnt+res_rd_cnt",
                          "+mgr_rf_cnt+mgr_rfd_cnt+mgr_rt_cnt+mgr_rr1_cnt+mgr_rr2_cnt+mgr_rd_cnt",
                          "+res_mgr_rr2_cnt+res_mgr_rd_cnt+res_rd_rr2_cnt",
                          "+rr1_rr2_cnt+rr1_rf_cnt+rr1_rfd_cnt+rr1_rt_cnt+rr1_rd_cnt",
                          "+rr2_rf_cnt+rr2_rfd_cnt+rr2_rt_cnt+rr2_rd_cnt",
                          "+rf_rfd_cnt+rf_rt_cnt+rf_rd_cnt+rfd_rt_cnt+rfd_rd_cnt",
                          "+rt_rd_cnt+rid_res_cnt",
                          "+rid_ev1+rid_ev2+rid_ev3+res_ev1+res_ev2+res_ev3+rid_ev4+rid_ev5+res_ev4+res_ev5",
                          "+rid_mgr_ev1+rid_mgr_ev2+rid_mgr_ev3+mgr_rid_ev1+mgr_rid_ev2+mgr_rid_ev3+rid_mgr_ev4+rid_mgr_ev5+mgr_rid_ev4+mgr_rid_ev5",
                          "+mgr_res_ev1+mgr_res_ev2+mgr_res_ev3+res_mgr_ev1+res_mgr_ev2+res_mgr_ev3+mgr_res_ev4+mgr_res_ev5+res_mgr_ev4+res_mgr_ev5"
)

In [ ]:
#sort the dataset so that training obs appear first
total<-total[order(total$split1),]

#create exp variables (leave one out average actual by categorical variable)
r_k<-0.2
total<-tv_spec_trans(total)


In [ ]:
#Extreme Randomized Trees 
mt<-model.matrix(as.formula(paste("y~", mdl_def_tree)), data=total)
ert1<-extraTrees(mt[total$split1==0,], total$y[total$split1==0], mtry=4, nodesize=10, numTreads=4, ntree=2000, numRandomCuts=6)


In [ ]:
#GLMNET 
m1<-model.matrix(as.formula(mdl_def_gnet), data=total)
filter_t<-total$split1==0
gnet1<-cv.glmnet(m1[filter_t,],total$ACTION[filter_t],family="binomial", alpha=0.8, lambda.min.ratio=0.0001)


In [ ]:
#GBM
t_frac<-with(total, sum(split1==0)/length(split1))
g2<-gbm(as.formula(paste("y~", mdl_def_tree)),weights=total$ws, #var.monotone=mono,
        data=total, train.fraction=t_frac, n.trees=1000, interaction.depth=8, n.minobsinnode=20, distribution="bernoulli", 
        shrinkage=0.03, bag.fraction=0.5, keep.data=F, verbose=T)


In [ ]:
#Random Forest
total$y_factor<-factor(total$y)
rf2<-randomForest(as.formula(paste("y_factor~", mdl_def_tree))
                  , data=total, subset=(total$split1==0), ntree=2000, mtry=3, nodesize=10, replace=F, 
                  , importance=TRUE, do.trace=10, maxnodes=500, sampsize=0.5*sum(total$split1==0), 
                  xtest=NULL,ytest=NULL,classwt=NULL)


In [ ]:
#noexp GBM@3000
g2_no_exp1<-gbm(as.formula(paste("y~", mdl_def_tree_noexp)), weights=total$ws, 
           data=total, train.fraction=t_frac, n.trees=3000, interaction.depth=8, n.minobsinnode=20, distribution="bernoulli", 
           shrinkage=0.03, bag.fraction=0.5, keep.data=F, verbose=T)



In [ ]:
#noexp ert@1000
mt_noexp<-model.matrix(as.formula(paste("y~", mdl_def_tree_noexp)), data=total)
ert_no_exp1<-extraTrees(mt_noexp[total$split1==0,], total$y[total$split1==0], mtry=3, nodesize=10
                 , numTreads=4, ntree=1000, numRandomCuts=6, evenCuts=F)


In [ ]:
#score all models
total$pred_ert<-predict(ert1, newdata=mt)
total$pred_gnet<-predict(gnet1, newx=m1, s="lambda.min", type="response")[,1]
total$pred_gbm<-predict(g2, newdata=total, type="response", n.trees=1000)
total$pred_rf<-predict(rf2, newdata=total, type="prob")[,2]
total$pred_gbm_noexp<-1-1/(1+exp(g2_no_exp1$fit))
total$pred_ert_noexp<-predict(ert_no_exp1, newdata=mt_noexp)

save(ert1, gnet1, g2, rf2, g2_no_exp1, ert_no_exp1, file="__final_models.RData")
